### Установите зависимости

Сначала нам нужно установить такие зависимости, как towhee, towhee.models и gradio upgrade pip.

In [ ]:
 pip install --upgrade pip

In [ ]:
 pip install -q towhee towhee.models gradio

### Подготовьте данные

Для использования механизма необходимо установить таблицу со статьями с источника [Sci-hub](https://sci-hub.ruhttps://sci-hub.ru).

In [ ]:
! curl -L -O https://sci-hub.ru/downloads/archives/5000.tab

### Создание коллекции Milvus

Перед началом работы, пожалуйста, убедитесь, что у вас запущен  [Milvus service](https://milvus.io/docs/install_standalone-docker.md). В блокноте используется [milvus 2.2.10](https://milvus.io/docs/v2.2.x/install_standalone-docker.md) and [pymilvus 2.2.11](https://milvus.io/docs/release_notes.md#2210).

In [ ]:
pip install -q pymilvus==2.2.11

In [ ]:
! pip install torch

In [ ]:
pip install towhee towhee.models gradio pandas pymilvus==2.2.11 transformers setuptools pyTorch

Затем определим функцию `cfp` для создания коллекции в Milvus, которая использует [L2 distance metric](https://milvus.io/docs/metric.md#Euclidean-distance-L2) и [IVF_FLAT index](https://milvus.io/docs/index.md#IVF_FLAT). Через configparserconfigparser парсируем ini файл Settings, в нем хранится значение Uri и токен к сервесу [ZilliZ](https://cloud.zilliz.com).

In [10]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility, MilvusClient
import configparser

cfp = configparser.RawConfigParser()
cfp.read('settings.ini')
connections.connect(
    uri= cfp.get("settings","uri"),
    token= cfp.get("settings","token")
)


Создаем функцию create_milvus_collection для создания схемы и коллекции данных.

In [ ]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility, MilvusClient
import configparser

def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
    
    fields = [
    FieldSchema(name='id', dtype=DataType.VARCHAR, descrition='ids', max_length=500, is_primary=True, auto_id=False),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, descrition='embedding vectors', dim=dim),
    FieldSchema(name='text', dtype=DataType.VARCHAR,max_length=1000)
    ]
    schema = CollectionSchema(fields=fields, description='reverse image search')
    collection = Collection(name=collection_name, schema=schema)

    # create IVF_FLAT index for collection.
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":2048}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection

collection = create_milvus_collection('testuser2', 768)

## Механизм Поиска

### Загрузить вопрос, встраиваемый в Milvus

Открываем заранее установленных файл со статьями и получаем строки, отделяем статьи от id. Получившийся текс необходимо деккомресовать и получить читабельный текс. Из-за большого объема текста данные не получется сразу отправить на сервер, для решения этой проблемы необходимо разбить текст на чанки.

In [ ]:
from base64 import b64decode
import requests
import base64
import zlib
import urllib

txt = open('5000.tab', 'r')
for i in range(5):
    line = txt.readline()
    txtId = line.split("\t")[0]
    src = line.split("\t")[1].replace("\n","")
    if len(src) < 1000:
        continue
    text2 = base64.b64decode(src)
    decompressed_data=zlib.decompress(text2, 16+zlib.MAX_WBITS)
    p =(
        pipe.input('id', 'text', 'question' )
        .map('question', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))
        .map('text','text',lambda x: x)
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map(('id', 'vec', 'text'), 'insert_status', ops.ann_insert.milvus_client(uri=cfp.get("settings","uri"), token=cfp.get("settings","token"), collection_name='testuser2'))
        .output()
        )
    

deco_date = decompressed_data.decode()
chunk_size = 78
chunks = [deco_date[i:(i + chunk_size)] for i in range(0, len(deco_date), chunk_size)]
print(deco_date,chunks)

In [ ]:
При помощи команды P создаем запрос на отправку данных на сервер. Операция DataCollection запущенная в цикле отправляет выбранные данные на сервер. 

In [ ]:
%%time
from towhee import pipe, ops
import numpy as np
from towhee.datacollection import DataCollection
p = (
    pipe.input('id', 'text' ,'question','answer')
        .map('question', 'vec', ops.text_embedding.dpr(model_name='facebook/dpr-ctx_encoder-single-nq-base'))
        .map('text','text',lambda x: x)
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map(('id', 'vec', 'text'), 'insert_status', ops.ann_insert.milvus_client(uri=cfp.get("settings","uri"), token=cfp.get("settings","token"), collection_name='testuser2'))
        .output()
)
for id, chunk in enumerate(chunks[:5]):
    
    DataCollection(p(f"{txtId}:{id}",chunk,chunk, txtId)).show()

In [ ]:
Print показывает количество данных отправленное на сервер

In [ ]:
print('Total number of inserted data is {}.'.format(collection.num_entities))

### Задайте вопрос Милвусу и Тови

После удачной отправки данных, создаем запрос получения статьи с id из фрагмента текста.

In [ ]:
%%time
collection.load()
ans_pipe = (
    pipe.input('question')
        .map('question', 'vec', ops.text_embedding.dpr(model_name="facebook/dpr-ctx_encoder-single-nq-base"))
        .map('vec', 'vec', lambda x: x / np.linalg.norm(x, axis=0))
        .map('vec', 'res', ops.ann_search.milvus_client(uri=cfp.get("settings","uri"), token=cfp.get("settings","token"), collection_name='testuser2', limit=1, **{'output_fields': ['id', 'text']}))
        .map('res', 'answer', lambda x: [x[0][0], x[0][3]])
        .output('question', 'answer')
)


ans = ans_pipe('tools, to borrow tools from them, and,')
ans = DataCollection(ans)
ans.show()

Проверка получившегося ответа

In [ ]:
ans[0]["answer"][1]